In [ ]:
from dash import dcc, html, Dash
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff

# Carregando o DataFrame
df = pd.read_csv('ecommerce_estatistica.csv')

# Gráfico 1: Histograma - Distribuição de Notas dos Produtos
fig1 = px.histogram(df,
                    x='Nota',
                    title='Histograma - Distribuição de Notas dos Produtos',
                    labels={'Nota': 'Nota', 'Frequência': 'Frequência'})

# Gráfico 2: Scatterplot - Correlação entre Nota e Preço
fig2 = px.scatter(df,
                  x='Nota',
                  y='Preço',
                  title='Scatterplot - Correlação entre Nota e Preço dos Produtos',
                  color_discrete_sequence=['#32a852'])

# Gráfico 3: Mapa de Calor - Correlação das Variáveis Numéricas
corr_cols = ['Nota', 'Preço', 'N_Avaliações', 'Desconto', 'Qtd_Vendidos_Cod', 'Material_Cod', 'Marca_Cod', 'Temporada_Cod']
df_corr = df[corr_cols].corr()
fig3 = px.imshow(df_corr,
                 text_auto=True,
                 aspect="auto",
                 title='Mapa de Calor - Correlação das Variáveis Numéricas',
                 color_continuous_scale='Viridis')

# Gráfico 4: Gráfico de Barra - Quantidade de Produtos por Temporada
temporada_cont = df['Temporada'].value_counts().reset_index()
temporada_cont.columns = ['Temporada', 'Quantidade']
fig4 = px.bar(temporada_cont,
              y='Temporada',
              x='Quantidade',
              orientation='h',  # Gráfico de barras horizontal
              title='Gráfico de Barra - Quantidade de Produtos por Temporada',
              color_discrete_sequence=['#325da8'])
fig4.update_layout(yaxis={'categoryorder': 'total ascending'}) # Ordena as barras

# Gráfico 5: Gráfico de Pizza - Distribuição de Produtos por Gênero
genero_reduzido = df['Gênero'].value_counts().nlargest(2)
fig5 = px.pie(values=genero_reduzido.values,
              names=genero_reduzido.index,
              title='Distribuição de Produtos por Gênero')

# Gráfico 6: Gráfico de Densidade - Quantidade de Itens Vendidos
qtd_vendidos_limpo = df['Qtd_Vendidos_Cod'].dropna()
hist_data = [qtd_vendidos_limpo]
group_labels = ['Quantidade de Vendas']
fig6 = ff.create_distplot(hist_data, group_labels,
                          show_hist=False, show_rug=False)
fig6.update_layout(title_text='Gráfico de Densidade - Quantidade de Itens Vendidos',
                   xaxis_range=[0, 60000])


# Gráfico 7: Gráfico de Regressão - Nota por Quantidade de Vendas
fig7 = px.scatter(df,
                  x='Nota',
                  y='Qtd_Vendidos_Cod',
                  trendline="ols",
                  title='Gráfico de Regressão - Nota por Quantidade de Vendas',
                  labels={'Qtd_Vendidos_Cod': 'Quantidade de Vendas'},
                  color_discrete_sequence=['orange'])
fig7.data[1].line.color = 'red'


# Configurando o App para visualização dos gráficos
app = Dash(__name__)

app.layout = html.Div(children=[
    html.H1(children='Dashboard de Análise de E-commerce'),

    html.Div(children='''
        Uma aplicação web para visualização de dados de um e-commerce de vestuário.
    '''),

    dcc.Graph(id='histograma-notas', figure=fig1),
    dcc.Graph(id='scatter-nota-preco', figure=fig2),
    dcc.Graph(id='heatmap-correlacao', figure=fig3),
    dcc.Graph(id='bar-temporada', figure=fig4),
    dcc.Graph(id='pie-genero', figure=fig5),
    dcc.Graph(id='density-vendas', figure=fig6),
    dcc.Graph(id='regressao-nota-vendas', figure=fig7),
])

# Executando o App
if __name__ == '__main__':
    app.run(debug=True)